<a href="https://colab.research.google.com/github/esheshka/gazprom/blob/main/gazprom_pred_30s_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np
import os

import scipy.ndimage
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

In [ ]:
l0 = 0
l1 = 200 + l0
app = 10000000

gazprom = open('gazprom_file070422.txt', 'r')
gp_pre = [float(float(i)*100) for i in gazprom.read().replace(',', '.').split()]
c = np.array([gp_pre[0:l1]])
l2 = len(gp_pre)
for i in range(l2 - l1):
    c = np.append(c, [gp_pre[1 + i:l1+1+i]], axis=0)

for c_f in os.listdir("c_fs"):
   with open(os.path.join("c_fs", c_f), 'r') as f:
        c_pre = [float(i) for i in f.read().split()]
        print(c_pre)
        l3 = len(c_pre)
        for i in range(l3 - l1):
            c = np.append(c, [c_pre[i:l1+i]], axis=0)


l1n = int(l1/10)
c = c[:, ::10]

c_post = np.array([(c[:, 0] - c[:, 1]) * app / c[:, 0]])
for i in range(l1n-2):
    c_post = np.append(c_post, [(c[:, i+1] - c[:, i+2]) * app / c[:, i+1]], axis=0)


c_f2 = c_post.T
df = pd.DataFrame(c_f2).astype('int')

change = 0
for i in range(l0 + 1):
    change += df[df.columns[-1 - i]].values
labels = (change)
labels_b = labels != 0
labels = labels[labels_b]

feature_matrix = df[df.columns[:-1 * l0 - 1]].values
feature_matrix = feature_matrix[labels_b, :]

In [ ]:
lab = scipy.ndimage.zoom(labels[0:250], 3)
plt.figure(figsize=(30, 10))
plt.plot(lab)
plt.show()

In [ ]:
train_feature_matrix, test_feature_matrix, train_labels, test_labels = train_test_split(feature_matrix, labels, test_size=0.4, random_state=42)

In [ ]:
clf2 = KNeighborsClassifier()
params2 = {
    'n_neighbors': np.arange(1, 11),
    'metric': ['manhattan', 'euclidean'],
    'weights': ['uniform', 'distance']
}

clf1 = RandomForestClassifier()

clf_grid = GridSearchCV(clf2, params2, cv=5, scoring='accuracy', n_jobs=-1)
clf_grid.fit(train_feature_matrix, train_labels)
clf1.fit(train_feature_matrix, train_labels)

In [ ]:
print(clf_grid.best_params_)
y_pred1 = clf1.predict(test_feature_matrix)
y_pred2 = clf_grid.predict(test_feature_matrix)


y_pred_bool1 = (y_pred1 > 0) == (test_feature_matrix[:, -1 * l0 - 1] > 0)
test_labels_bool1 = (test_labels > 0) == (test_feature_matrix[:, -1 * l0 - 1] > 0)

y_pred_bool2 = (y_pred2 > 0) == (test_feature_matrix[:, -1 * l0 - 1] > 0)
test_labels_bool2 = (test_labels > 0) == (test_feature_matrix[:, -1 * l0 - 1] > 0)

print(accuracy_score(test_labels, y_pred1))
print(accuracy_score(test_labels_bool1, y_pred_bool1))
print(accuracy_score(test_labels[y_pred1 > 0] > 0, y_pred1[y_pred1 > 0] > 0))

print(accuracy_score(test_labels, y_pred2))
print(accuracy_score(test_labels_bool2, y_pred_bool2))
print(accuracy_score(test_labels[y_pred2 > 0] > 0, y_pred2[y_pred2 > 0] > 0))

alpha = 2000
# Ответы с точностью до alpha
ep = np.append([test_labels[(y_pred1 > test_labels - alpha) * (y_pred1 < test_labels + alpha)]], [y_pred1[(y_pred1 > test_labels - alpha) * (y_pred1 < test_labels + alpha)]], axis=0)
print("Точность alpha - ", ep.shape[1] / test_labels.shape[0])
print(ep.T)
print(np.append([test_labels[y_pred2 == test_labels]], [y_pred2[y_pred2 == test_labels]], axis=0).T)
print(np.append([test_labels[y_pred1 > 0]], [y_pred1[y_pred1 > 0]], axis=0).T)
print(np.append([test_labels[y_pred2 > 0]], [y_pred2[y_pred2 > 0]], axis=0).T)